In [142]:
import pandas as pd
import numpy as np

In [143]:
# 출력 옵션 변경

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1300)

In [144]:
dlvr = pd.read_csv('../data/dlvr_call.csv');

c:\users\jeaung-lee\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [145]:
dlvr.shape

(4296271, 32)

In [146]:
dlvr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4296271 entries, 0 to 4296270
Data columns (total 32 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   SN                         int64 
 1   PROCESS_DT                 object
 2   DLVR_REQUST_ENTRPS_ID      object
 3   DLVR_REQUST_STTUS_VALUE    int64 
 4   DLVR_RCEPT_ENTRPS_ID       object
 5   DLVR_STORE_ID              object
 6   DLVR_STORE_INDUTY_NM       object
 7   DLVR_STORE_LEGALDONG_CODE  int64 
 8   DLVR_STORE_BRTC_NM         object
 9   DLVR_STORE_SIGNGU_NM       object
 10  DLVR_STORE_LEGALDONG_NM    object
 11  DLVR_STORE_LI_NM           object
 12  DLVR_STORE_ADSTRD_CODE     int64 
 13  DLVR_STORE_RDNMADR_CODE    object
 14  DLVR_DSTN_LEGALDONG_CODE   int64 
 15  DLVR_DSTN_BRTC_NM          object
 16  DLVR_DSTN_SIGNGU_NM        object
 17  DLVR_DSTN_LEGALDONG_NM     object
 18  DLVR_DSTN_LI_NM            object
 19  DLVR_DSTN_ADSTRD_CODE      object
 20  DLVR_DSTN_RDNMADR_CODE  

In [172]:
# 지역을 서울로 한정

dlvr = dlvr[dlvr['DLVR_DSTN_BRTC_NM'] == '서울특별시']

In [173]:
# 지역, 업종, 매출 컬럼 선택
# 행정동코드 ['DLVR_DSTN_ADSTRD_CODE'], 배달상점 업종이름 (카테고리) ['DLVR_STORE_INDUTY_NM'], 배달상품금액 ['GOODS_AMOUNT']

df_dlvr = dlvr[['DLVR_DSTN_ADSTRD_CODE','DLVR_STORE_INDUTY_NM','GOODS_AMOUNT']]

In [174]:
# 행정동 코드가 이상한 행 삭제  

df_dlvr = df_dlvr[df_dlvr.DLVR_DSTN_ADSTRD_CODE != '\\N']
df_dlvr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1018134 entries, 0 to 4296270
Data columns (total 3 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   DLVR_DSTN_ADSTRD_CODE  1018134 non-null  object
 1   DLVR_STORE_INDUTY_NM   1018134 non-null  object
 2   GOODS_AMOUNT           1018134 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 31.1+ MB


In [149]:
# 행정동코드에 00이 붙어있어 제거

# df_dlvr['DLVR_DSTN_ADSTRD_CODE'] = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].astype('int64')/100

In [175]:
# 업종 컬럼 값 확인

df_dlvr['DLVR_STORE_INDUTY_NM'].unique()

array(['족발/보쌈', '패스트푸드', '분식', '치킨', '회', '아시안/양식', '한식', '피자', '돈까스/일식',
       '카페/디저트', '찜탕', '중식', '야식', '배달전문업체', '심부름', '도시락'], dtype=object)

In [176]:
# 업종이'배달전문업체','심부름'인 행 삭제

df_dlvr.replace(['배달전문업체','심부름'], np.nan, inplace=True)
df_dlvr = df_dlvr.dropna(axis=0) 

In [177]:
df_dlvr['DLVR_STORE_INDUTY_NM'].unique()

array(['족발/보쌈', '패스트푸드', '분식', '치킨', '회', '아시안/양식', '한식', '피자', '돈까스/일식',
       '카페/디저트', '찜탕', '중식', '야식', '도시락'], dtype=object)

### 지역별 업종 매출 순위 

In [178]:
area = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].unique()
# area = [1159066000]
df = pd.DataFrame(columns=['DLVR_DSTN_ADSTRD_CODE','DLVR_STORE_INDUTY_NM','GOODS_AMOUNT'])

for a in area:
    df_area = df_dlvr[df_dlvr['DLVR_DSTN_ADSTRD_CODE'] == a]
    induty = df_area['DLVR_STORE_INDUTY_NM'].unique()    
    for i in induty:
        df_induty = df_area[df_area['DLVR_STORE_INDUTY_NM'] == i]
        indSum = sum(df_induty['GOODS_AMOUNT'])
        dic = { 'DLVR_DSTN_ADSTRD_CODE' : a, 'DLVR_STORE_INDUTY_NM' : i,'GOODS_AMOUNT' : indSum}
        df = df.append(dic,ignore_index=True)

In [179]:
df1 = df.copy() 

In [180]:
# 행정동코드 열 정수형으로 변경(데이터 표기 통일)

df1['DLVR_DSTN_ADSTRD_CODE'] = df1['DLVR_DSTN_ADSTRD_CODE'].astype('int64')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3673 entries, 0 to 3672
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DLVR_DSTN_ADSTRD_CODE  3673 non-null   int64 
 1   DLVR_STORE_INDUTY_NM   3673 non-null   object
 2   GOODS_AMOUNT           3673 non-null   object
dtypes: int64(1), object(2)
memory usage: 86.2+ KB


In [181]:
# 상품금액 합게가 0인 행 삭제
# 상품금액 열 정수형으로 변경

df1.replace(0, np.nan, inplace=True)
df1 = df1.dropna(axis=0)
df1['GOODS_AMOUNT'] = df1['GOODS_AMOUNT'].astype('int64')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3558 entries, 0 to 3672
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DLVR_DSTN_ADSTRD_CODE  3558 non-null   int64 
 1   DLVR_STORE_INDUTY_NM   3558 non-null   object
 2   GOODS_AMOUNT           3558 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 111.2+ KB


In [182]:
df_group_top3 = df1.sort_values(by='GOODS_AMOUNT', ascending=False).groupby('DLVR_DSTN_ADSTRD_CODE', sort=False).head(3)
df_group = df_group_top3.groupby('DLVR_DSTN_ADSTRD_CODE',as_index=False)

In [19]:
print(df_group.first())

     DLVR_DSTN_ADSTRD_CODE DLVR_STORE_INDUTY_NM  GOODS_AMOUNT
0                 11110560                    회         64000
1                 11140550               돈까스/일식         56900
2                 11170520                   분식         14000
3                 11215730                    회         21000
4                 11215740                   치킨          2000
5                 11230536                   찜탕         26000
6                 11260580               카페/디저트        119800
7                 11260600               카페/디저트        152700
8                 11260610               카페/디저트         75900
9                 11260620               카페/디저트       3278500
10                11260630               카페/디저트       1311800
11                11260680               카페/디저트        724700
12                11260690               카페/디저트        910100
13                11290525               카페/디저트         39000
14                11290555               카페/디저트         57100
15      

In [104]:
df_group_top3.to_csv('../data/df_group.csv')

### 행정동별 TOP3 추출 완료.

### 행정동별 생활인구 데이터 전처리

In [48]:
pop1 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_201912.csv',header=0, index_col=False);
pop2 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202001.csv',header=0, index_col=False);
pop3 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202002.csv',header=0, index_col=False);
pop4 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202003.csv',header=0, index_col=False);
pop5 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202004.csv',header=0, index_col=False);
pop6 = pd.read_csv('../data/LOCAL_PEOPLE_DONG_202005.csv',header=0, index_col=False);

In [49]:
# 2019.12 ~ 2020.05 행정동별 생활인구 데이터 병합
pop = pd.concat([pop1,pop2,pop3,pop4,pop5,pop6])

In [197]:
df_pop = pop.copy()

In [198]:
df_pop = df_pop.drop(['시간대구분','총생활인구수'], axis=1)
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1862208 entries, 0 to 315455
Data columns (total 30 columns):
 #   Column           Dtype  
---  ------           -----  
 0   기준일ID            int64  
 1   행정동코드            int64  
 2   남자0세부터9세생활인구수    float64
 3   남자10세부터14세생활인구수  float64
 4   남자15세부터19세생활인구수  float64
 5   남자20세부터24세생활인구수  float64
 6   남자25세부터29세생활인구수  float64
 7   남자30세부터34세생활인구수  float64
 8   남자35세부터39세생활인구수  float64
 9   남자40세부터44세생활인구수  float64
 10  남자45세부터49세생활인구수  float64
 11  남자50세부터54세생활인구수  float64
 12  남자55세부터59세생활인구수  float64
 13  남자60세부터64세생활인구수  float64
 14  남자65세부터69세생활인구수  float64
 15  남자70세이상생활인구수     float64
 16  여자0세부터9세생활인구수    float64
 17  여자10세부터14세생활인구수  float64
 18  여자15세부터19세생활인구수  float64
 19  여자20세부터24세생활인구수  float64
 20  여자25세부터29세생활인구수  float64
 21  여자30세부터34세생활인구수  float64
 22  여자35세부터39세생활인구수  float64
 23  여자40세부터44세생활인구수  float64
 24  여자45세부터49세생활인구수  float64
 25  여자50세부터54세생활인구수  float64
 26  여자55세부터59세생활인구수  float64
 27  여자60세부터64세생활인

### 행정동별 일별 생활인구

In [215]:
df_dlvr['DLVR_DSTN_ADSTRD_CODE'] = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].astype('int64')/100

NameError: name 'df_dlvr1' is not defined

In [216]:
area = df_dlvr['DLVR_DSTN_ADSTRD_CODE'].unique()
col_name = list(df_pop.columns.values)
df_daily_pop = pd.DataFrame(columns = col_name)

for a in area:
    df_area = df_pop[df_pop['행정동코드'] == int(a)]
    df_area_daily = df_area.groupby(df_area['기준일ID'], as_index=False).sum()
    df_area_daily['행정동코드'] = int(a)    
    df_daily_pop = df_daily_pop.append(df_area_daily)    

In [217]:
# df_daily_pop = 행정동별 일별 생활인구
df_daily_pop

,기준일ID,행정동코드,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,남자40세부터44세생활인구수,남자45세부터49세생활인구수,남자50세부터54세생활인구수,남자55세부터59세생활인구수,남자60세부터64세생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,여자0세부터9세생활인구수,여자10세부터14세생활인구수,여자15세부터19세생활인구수,여자20세부터24세생활인구수,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20191201,11590660,25958.1085,19953.9611,30282.4308,34637.9595,44281.0987,35084.4635,36715.9141,35467.4223,43197.9349,34747.7257,30369.1048,23723.4539,17874.2152,38502.4389,26771.2644,20547.1290,31106.4429,39269.4243,41137.9775,33179.2245,37761.3045,36441.5078,41949.9068,33458.7692,32729.5385,29777.4467,24406.5111,66407.4056
1,20191202,11590660,30704.6334,23602.6081,41738.1643,34562.4936,41328.7065,32941.1219,34126.0494,33720.1075,39401.8654,32212.6066,28652.8222,22505.4320,17298.9260,37286.5527,32394.6555,24863.1187,33324.2631,38620.3151,39847.5291,32522.3121,37329.4102,36997.5988,43383.1757,34311.1245,33953.6908,29667.5304,23851.1687,66806.6710
2,20191203,11590660,31902.0476,24523.0589,41222.0171,34661.0202,40214.2814,33045.3086,34699.1899,34622.4411,39269.6401,30811.0926,29810.8610,22689.6308,17267.9512,38058.1916,31494.2439,24172.0476,31195.9397,37701.4871,40531.9630,32562.0338,36875.5968,37740.1790,43516.1347,34013.4097,34080.6785,30877.5882,23522.7498,66219.2662
3,20191204,11590660,31924.2087,24540.0963,41666.2431,34377.0813,40658.4741,32800.5905,33760.4731,33756.0083,39674.0386,31588.1149,28440.5940,22706.2451,17414.7745,37311.5743,31275.9305,24004.4916,33434.2887,37337.7316,40407.1333,32522.9367,37225.8520,37753.5437,43685.5198,34931.2871,33735.1826,30666.3995,24280.1281,66629.7517
4,20191205,11590660,31958.5472,24566.4915,41577.7695,33380.5341,41446.2486,32305.7966,33732.0523,33708.8770,39817.3354,30891.0397,29440.5692,23180.2833,17012.6012,37111.1811,31964.3651,24532.8676,31571.5922,38062.2496,39920.6682,32260.4140,37439.0830,37212.1138,42646.7599,34282.8788,33001.3318,29534.7890,23072.0897,67011.6032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,20200527,11740580,21313.8609,7690.2592,6953.7566,8160.1002,11899.1264,13524.3414,18032.0009,15662.9658,17931.8734,12412.8755,13600.5050,13079.6280,8911.4342,18255.8097,20824.9950,7656.6980,8217.5007,11452.8577,13087.6007,17926.2862,21083.3780,18526.4818,16290.9420,15755.5883,16639.8281,15924.6774,12650.7295,27159.3059
179,20200528,11740580,19950.5322,7198.3563,6591.9563,8225.6346,11138.8170,13425.6507,16951.3961,15016.0531,16619.1326,11913.3472,13293.3193,12221.7388,8314.2197,17731.6704,19360.8889,7118.3909,8008.6241,10580.7866,12665.5123,16630.4314,21035.5184,17965.8673,16687.8727,15464.4339,16211.6881,15673.4397,12330.0560,27483.3662
180,20200529,11740580,20458.2015,7381.5291,7624.7773,8413.5441,11595.6192,13822.3885,18238.0252,16425.9230,17344.0204,12325.2933,14082.1998,13071.8047,8568.8640,17579.0446,20829.0624,7658.1948,8301.1617,11248.4909,13016.7245,16675.9733,22081.5877,17705.4756,16495.9292,16005.4038,15847.7328,15580.8231,12269.5288,27886.2479
181,20200530,11740580,19902.5133,7181.0309,7739.1794,9563.1985,12746.4126,15843.5807,21916.1269,18659.3488,20449.3745,13635.6170,15551.5091,13517.0584,8711.8936,18750.4516,20513.7934,7542.2799,9256.3270,12054.6533,14466.3459,19241.7393,22887.1413,19263.9312,17230.5919,16199.5428,16657.9062,15661.3956,12430.5485,27665.5214


In [100]:
df_daily_pop.to_csv('../data/df_daily_pop.csv')

### 행정동별 생활인구(전체기간 총합)

In [199]:
df_pop

,기준일ID,행정동코드,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,남자40세부터44세생활인구수,남자45세부터49세생활인구수,남자50세부터54세생활인구수,남자55세부터59세생활인구수,남자60세부터64세생활인구수,남자65세부터69세생활인구수,남자70세이상생활인구수,여자0세부터9세생활인구수,여자10세부터14세생활인구수,여자15세부터19세생활인구수,여자20세부터24세생활인구수,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20191201,11215710,452.3937,208.7971,1267.7678,5642.9235,5433.6801,2494.5993,1691.4831,1042.7096,1058.3378,895.4639,875.9212,601.9097,496.2165,850.6820,388.4005,180.3721,1586.7039,6973.6176,4259.7022,2036.1446,1285.4200,794.6241,833.7179,772.1417,773.2152,827.1455,565.5378,1405.3707
1,20191201,11680545,1298.5585,731.2901,889.5126,895.5983,1397.2099,1609.7025,1825.4062,1741.6219,1829.6326,1370.2235,1124.6595,878.5269,695.3232,1717.7044,1435.3327,787.0444,1130.2773,1215.9905,1628.3736,1503.9461,1752.1118,1591.0488,1555.4675,1033.6247,1094.5083,988.0638,944.5295,2710.1619
2,20191201,11680610,1083.2150,1559.2566,2339.8178,991.4434,990.1854,946.7321,1035.0291,1390.5989,2083.7386,1739.9398,1523.3575,1055.0147,899.4087,1534.7962,952.8383,1059.4625,1693.2841,959.5733,980.3396,888.9359,997.1139,1738.2101,2423.0246,1408.2536,1184.3355,1071.2366,858.9868,1878.8251
3,20191201,11680655,1191.0306,607.5946,639.2233,608.4116,697.5009,663.3373,919.4319,1074.9392,1228.4840,994.8977,765.8304,794.3326,659.9513,1110.7365,1067.8148,642.6462,862.7861,681.7974,767.0526,876.7431,1119.2400,1051.8991,1274.9860,852.3075,933.6367,949.1604,637.7794,1557.6550
4,20191201,11710570,1586.6273,818.9047,1204.9034,1172.2571,1195.7198,1163.9396,1472.8633,1347.5375,1603.9937,1482.1456,1575.2363,1455.9243,1080.4173,1650.7427,1459.9421,768.2610,1166.6483,1153.8815,1276.9370,1321.7052,1625.5710,1434.9812,1861.6645,1766.8905,1991.8166,1977.4260,1332.3513,2779.2676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315451,20200531,11500590,1462.5609,650.6488,751.8066,847.6573,1457.5636,1692.3361,1832.7683,1294.5065,1412.5338,989.9783,948.6449,827.1785,679.5684,1484.4242,1518.1599,587.3533,669.4936,1187.2936,1628.3094,1521.2461,1579.0513,1445.1335,1450.2348,1115.3927,1360.6946,1224.2473,1002.7457,2228.7270
315452,20200531,11710520,1522.6487,720.0644,783.0475,712.6163,1304.9236,1243.4551,1534.7862,1333.4719,1432.0687,1172.1130,1177.2657,1123.2123,835.0033,1543.8213,1327.6419,672.1250,931.3391,1293.4632,1699.0453,1569.7967,1621.0363,1411.4797,1646.5681,1158.6589,1462.9537,1414.7024,966.1372,2202.4438
315453,20200531,11410585,633.9095,199.2584,749.1463,3094.7758,2890.4101,1641.4097,1340.0429,829.3566,876.4381,751.4725,866.5132,649.7455,398.1159,1141.3060,309.0596,155.5619,1184.6523,6133.0721,3783.0008,1821.0791,1273.7274,873.9549,836.7297,839.4791,766.9024,733.7818,567.0416,1604.2628
315454,20200531,11410565,495.1475,317.0280,619.4652,953.5512,1081.7686,879.9429,1039.5685,842.3417,1135.0484,828.5651,862.8410,797.5366,500.6617,1076.5711,563.9444,356.7550,591.8719,1046.6147,1305.8847,938.6720,1057.4920,849.3335,915.2439,896.5478,981.3186,863.6758,516.9980,1682.2935


In [205]:
area = df_dlvr1['DLVR_DSTN_ADSTRD_CODE'].unique()
col_name = list(df_pop.columns.values)
df_total_pop = pd.DataFrame(columns = col_name)

for a in area:
    df_area = df_pop[df_pop['행정동코드'] == int(a)]
    df_area_total = df_area.sum()
    df_area_total['행정동코드'] = int(a)*100  
    df_total_pop = df_total_pop.append(df_area_total, ignore_index=True)

In [207]:
# 기준일ID는 의미없어 삭제
df_total_pop = df_total_pop.drop(['기준일ID'], axis=1)

In [212]:
df_total_pop['행정동코드'] = df_total_pop['행정동코드'].astype('int64')

In [213]:
# df_total_pop = 행정동별 생활인구(6개월 누적)
df_total_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   행정동코드            256 non-null    int64  
 1   남자0세부터9세생활인구수    256 non-null    float64
 2   남자10세부터14세생활인구수  256 non-null    float64
 3   남자15세부터19세생활인구수  256 non-null    float64
 4   남자20세부터24세생활인구수  256 non-null    float64
 5   남자25세부터29세생활인구수  256 non-null    float64
 6   남자30세부터34세생활인구수  256 non-null    float64
 7   남자35세부터39세생활인구수  256 non-null    float64
 8   남자40세부터44세생활인구수  256 non-null    float64
 9   남자45세부터49세생활인구수  256 non-null    float64
 10  남자50세부터54세생활인구수  256 non-null    float64
 11  남자55세부터59세생활인구수  256 non-null    float64
 12  남자60세부터64세생활인구수  256 non-null    float64
 13  남자65세부터69세생활인구수  256 non-null    float64
 14  남자70세이상생활인구수     256 non-null    float64
 15  여자0세부터9세생활인구수    256 non-null    float64
 16  여자10세부터14세생활인구수  256 non-null    float64
 17  여자15세부터19세생활인구수 

In [214]:
df_total_pop.to_csv('../data/df_total_pop.csv')

### 행정동코드 좌표 데이터

In [159]:
geo = pd.read_excel('../data/행정_법정동 중심좌표.xlsx',header=0, index_col=False);

In [185]:
area = dlvr['DLVR_DSTN_ADSTRD_CODE'].unique()
dlvrArea = geo.loc[:,'코드'].isin(area) 

In [186]:
geo = geo[dlvrArea]

In [ ]:
geo.to_csv('../data/geo.csv')

## 배달 데이터 + 위치정보 병합

In [187]:
df_group_top3.rename({'DLVR_DSTN_ADSTRD_CODE':'코드'}, axis=1, inplace=True)


In [188]:
geo

,코드,시도,시군구,읍면동,하위,위도,경도,코드종류
4,1111056000,서울특별시,종로구,평창동,NaN,37.613029,126.974485,H
15,1111070000,서울특별시,종로구,숭인제1동,NaN,37.573172,126.979220,H
33,1117052000,서울특별시,용산구,용산2가동,NaN,37.542884,126.985008,H
58,1120064500,서울특별시,성동구,옥수동,NaN,37.540788,127.020624,H
61,1120067000,서울특별시,성동구,성수2가제1동,NaN,37.540855,127.056517,H
62,1120069000,서울특별시,성동구,성수2가제3동,NaN,37.540855,127.056517,H
66,1121573000,서울특별시,광진구,군자동,NaN,37.553038,127.073657,H
67,1121574000,서울특별시,광진구,중곡제1동,NaN,37.538210,127.082191,H
72,1121581000,서울특별시,광진구,광장동,NaN,37.547835,127.104695,H
80,1123053600,서울특별시,동대문구,용신동,NaN,37.575868,127.037305,H


In [183]:
df_group_top3

,DLVR_DSTN_ADSTRD_CODE,DLVR_STORE_INDUTY_NM,GOODS_AMOUNT
192,1153077000,치킨,224861112
328,1153078000,치킨,200755506
108,1153074000,치킨,192852393
269,1153075000,치킨,162191902
202,1154551000,치킨,158554706
18,1153079000,치킨,142240951
495,1154561000,치킨,133675224
29,1153073000,치킨,128224307
193,1153077000,한식,117560400
324,1153078000,족발/보쌈,117488844


In [189]:
df_dlvr_geo = pd.merge(df_group_top3, geo, on='코드', how='left')

In [195]:
df_dlvr_geo.to_csv('../data/df_dlvr_geo.csv')

In [193]:
df_dlvr_geo

,코드,DLVR_STORE_INDUTY_NM,GOODS_AMOUNT,시도,시군구,읍면동,하위,위도,경도,코드종류
0,1153077000,치킨,224861112,서울특별시,구로구,오류제1동,NaN,37.495650,126.887771,H
1,1153078000,치킨,200755506,서울특별시,구로구,오류제2동,NaN,37.495650,126.887771,H
2,1153074000,치킨,192852393,서울특별시,구로구,개봉제1동,NaN,37.495650,126.887771,H
3,1153075000,치킨,162191902,서울특별시,구로구,개봉제2동,NaN,37.495650,126.887771,H
4,1154551000,치킨,158554706,서울특별시,금천구,가산동,NaN,37.476681,126.883767,H
5,1153079000,치킨,142240951,서울특별시,구로구,수궁동,NaN,37.493837,126.831527,H
6,1154561000,치킨,133675224,서울특별시,금천구,독산제1동,NaN,37.456979,126.895656,H
7,1153073000,치킨,128224307,서울특별시,구로구,고척제2동,NaN,37.495650,126.887771,H
8,1153077000,한식,117560400,서울특별시,구로구,오류제1동,NaN,37.495650,126.887771,H
9,1153078000,족발/보쌈,117488844,서울특별시,구로구,오류제2동,NaN,37.495650,126.887771,H
